# Premium ground vehicles

## Gather wiki links

Caches links in a text file

In [ ]:
import os
from utils import generate_premium_ground_vehicle_links, dump_list_to_file

outfile = 'data/premium_ground_vehicle_links.txt'

pgv_links = generate_premium_ground_vehicle_links()
pgv_links_file_path = os.path.join(os.path.abspath(''), outfile)
dump_list_to_file(pgv_links, pgv_links_file_path)

with open(pgv_links_file_path) as f:
    lines = f.read().split('\n')

lines

## Gather ground vehicle information

The following vehicle information will be gathered (tracking with ✅/❌):
- ❌ Vehicle name
- ❌ Country
- ❌ Type of tank
- ❌ Rank
- ❌ BR for each game mode
- ❌ Cost (pull from "Purchase" on page, if exists)
- ❌ Wheels vs treads
- ❌ Hull armor (front/side/back)
- ❌ Turret armor (front/side/back)
- ❌ Crew members
- ❌ Visibility
- ❌ Horizontal guidance
- ❌ Vertical guidance
- ❌ Is amphibious
- ❌ Forward speed (AB)
- ❌ Forward speed (RB/SB)
- ❌ Back speed (AB)
- ❌ Back speed (RB/SB)
- ❌ Engine power (AB)
- ❌ Engine power (RB/SB)
- ❌ Power-to-weight ratio (AB)
- ❌ Power-to-weight ratio (RB/SB)
- ❌ Weight (tons)
- ❌ Repair cost (AB)
- ❌ Repair cost (RB/SB)
- ❌ Crew training
- ❌ Crew training (Expert)
- ❌ Crew training (Aces)
- ❌ Crew training (Research Aces)
- ❌ Modifications list
- ❌ First stage ammunition amount (maybe?)
- ❌ Reload time
- ❌ Max ammo
- ❌ Has stabilizer
- ❌ Fire rate
- ❌ Ammunitions
  - ❌ name
  - ❌ type
  - ❌ pen @ 10m
  - ❌ pen @ 100m
  - ❌ pen @ 500m
  - ❌ pen @ 1000m
  - ❌ pen @ 1500m
  - ❌ pen @ 2000m
  - ❌ projectile velocity
  - ❌ projectile mass
  - ❌ fuse delay
  - ❌ fuse sensitivity
  - ❌ explosive mass
  - ❌ degrees richochet 0% chance 
  - ❌ degrees richochet 50% chance 
  - ❌ degrees richochet 100% chance 
- ❌ coax machine gun caliber
- ❌ has mounted MG